# Getting Started with LangChain

Covers fundamental LangChain components for chaining LLM prompts and tools.

In [3]:
!pip install langchain
!pip install -qU langchain-groq
!pip install -qU langchain-openai
!pip install python_dotenv

## Getting started

In [4]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
import os

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
gpt4o = ChatOpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
    model_name = "gpt-4o-mini")

In [7]:
messages = [
    (
        "system",
        "Sei un chatbot assistente di cucina. Aiuta l'utente con le ricette.",
    ),
    ("human", "A che temperatura cuoce la pizza?"),
]

gpt_ping = gpt4o.invoke(messages)
print(gpt_ping)

content='La temperatura ideale per cuocere la pizza varia in base al tipo di forno che stai utilizzando. In generale:\n\n- **Forno di casa**: Se usi un forno tradizionale, è consigliabile preriscaldarlo a circa 250-280°C. Questo aiuta a ottenere una base croccante e una cottura uniforme.\n  \n- **Forno a legna**: Questi forni possono raggiungere temperature molto più elevate, intorno ai 400-500°C. La pizza in questo tipo di forno cuoce molto rapidamente, in soli 90 secondi a 3 minuti.\n\n- **Forno elettrico o a gas**: Anche in questo caso, è meglio impostare la temperatura su 250-300°C per risultati ottimali.\n\nRicorda di utilizzare una pietra refrattaria se possibile, poiché aiuta a mantenere e distribuire il calore in modo uniforme, rendendo la pizza ancora più deliziosa!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 38, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_toke

## Prompting Template

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Dimmi tre razze appartenti a alla specie {specie}")

prompt_template.invoke({"specie": "squali"})

StringPromptValue(text='Dimmi tre razze appartenti a alla specie squali')

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Sei un chatbot esperto di zoologia"),
    ("user", "Dimmi tre razze appartenti a alla specie {specie}")
])

prompt_template.invoke({"specie": "dinosauri"})

messages=[SystemMessage(content='Sei un chatbot esperto di zoologia', additional_kwargs={}, response_metadata={}), HumanMessage(content='Dimmi tre razze appartenti a alla specie dinosauri', additional_kwargs={}, response_metadata={})]


## LangChain Expression Language (LCEL)

Double prompt to improve response

In [15]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai la piscina?
User: Era molto piccola e troppo calda.
"""

standard_response = """ Grazie mille per il tuo feedback. Faremo il nostro meglio per migliorarci in futuro.
"""

PROMPT_INTERACTION_1 = """ Sei un chatbot assistente in un servizio clienti di un hotel. Stai avendo una conversazione con un cliente:
```{chat_history}```.
Genera una risposta appropriata per continuare la conversazione, adattando questa risposta standard:
Risposta standard: ```{standard_response}```.
"""

PROMPT_REFINE_1 = """ Sei un chatbot assistente in un servizio clienti di un hotel. Stai avendo una conversazione con un cliente.
La tua ultima risposta è stata questa ```{first_response}```.
Elimina eventuali domande e assicurati che sia simile a questa risposta standard: ```{standard_response}```.
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response"], template=PROMPT_INTERACTION_1)
prompt_clean_questions = PromptTemplate(input_variables=["first_response","standard_response"], template=PROMPT_REFINE_1)
std_resp = PromptTemplate.from_template(standard_response)

chain_one = prompt_start | gpt4o | StrOutputParser()
chain_two = prompt_clean_questions | gpt4o | StrOutputParser()

final_chain =({"first_response" : chain_one, "standard_response": std_resp}
        | RunnablePassthrough.assign(final_response = chain_two))

result = final_chain.invoke({"chat_history": chat_history, "standard_response": standard_response})

print(result['final_response'])

```text=' Grazie mille per il tuo feedback. Faremo il nostro meglio per migliorarci in futuro.\n'```


Customizing focus of response

In [ ]:
chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai?
User: Era molto piccola e troppo calda.
Chatbot: In che modo pensi potremmo migliorarci la prossima volta?
User: Un servizio di prenotazioni aiuterebbe magari
"""

standard_response = """ Grazie per il tuo feedback, ottimo consiglio.
Faremo il possibile per migliorare il servizio piscina, ci spiace non sia stato all'altezza delle tue aspettative! """

user_first_impression = """Era molto piccola e troppo calda. """
user_goal = """Un servizio di prenotazioni aiuterebbe magari"""

PROMPT_INTERACTION_F = """Sei un chatbot assistente che sta aiutando un cliente insoddisfatto riguardo a un servizio dell'hotel.
Stai avendo una conversazione: ```{chat_history}```.
Genera una risposta appropriata per continuare questa conversazione, adattando questa risposta standard.
Risposta standard: ```{standard_response}```
L'utente ha detto questo sull'esperienza: {user_first_impression}:
L'utente ti ha detto che spera di vedere miglioramenti: ```{user_goal}```
Scrivi solo la risposta.
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response", "user_first_impression", "user_goal"], template=PROMPT_INTERACTION_F)

chain_one = prompt_start | gpt4o | StrOutputParser()

result = chain_one.invoke({"chat_history": chat_history, "standard_response": standard_response,
                           "user_first_impression": user_first_impression, "user_goal": user_goal })

print(result)

In [ ]:

chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai?
User: Era molto piccola e troppo calda.
Chatbot: In che modo pensi potremmo migliorarci la prossima volta?
User: Un servizio di prenotazioni aiuterebbe magari
"""

user_first_impression = """Era molto piccola e troppo calda. """
user_response = """Un servizio di prenotazioni aiuterebbe magari"""


PROMPT_GOAL = """Sei un chatbot assistente che sta aiutando un cliente insoddisfatto riguardo a un servizio dell'hotel.
Stai avendo una conversazione: ```{chat_history}```.
Qual è lo scopo dell'utente? In che modo aiuterebbe a migliorare?
```{user_response}```
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response", "user_response"], template=PROMPT_GOAL)

chain_one = prompt_start | llama3 | StrOutputParser()

user_goal = chain_one.invoke({"chat_history": chat_history, "user_response": user_response })

print(user_goal)